# GPTでレビュー分類

In [81]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [ ]:
# データのパス
data_path = 'data/raw/android.json'

# データ読み込み
with open(data_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    
df = pd.DataFrame(data['data'])

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

In [83]:
import openai
import os

# OpenAI APIのキーを環境変数から取得
openai.api_key = os.getenv("OPENAI_API_KEY")

#print(os.getenv("OPENAI_API_KEY"))  # APIキーが表示されれば成功

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

# テスト用の文章を生成
try:
    chat_completion = client.chat.completions.create(
        model="gpt-4o",  # 正しいモデル名
        messages=[
            {
                "role": "system",
                "content": "レビューの重要な情報を保持しながら、ポジティブな意見とネガティブな意見に分類しクリーニングしてください。",
            },
            {
                "role": "user",
                "content": "レビュー: このアプリ、まあまあだけどy使いにくい😄部分もあります。",
            }
        ]
    )
    # 応答内容を表示
    print(chat_completion.choices[0].message.content)
except openai.error.OpenAIError as e:
    print(f"OpenAI API エラー: {e}")

In [ ]:
# GPTを用いたクリーニング用の関数を定義

def clean_review_with_gpt(review):
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-4o",  # 正しいモデル名
            messages=[
            {
                "role": "system",
                "content": "レビューの重要な情報を出来るだけ保持しながら、ポジティブな意見とネガティブな意見に分類してください。その際，必要であればクリーニングしてください。特にポジティブな意見またはネガティブな意見がない場合は「特になし」と答えてください。",
            },
            {
                "role": "user",
                "content": f"レビュー: {review}",
            }
            ]
        )
        return chat_completion.choices[0].message.content
    except openai.error.OpenAIError as e:
        print(f"OpenAI API エラー: {e}")
        return None
    
# テスト用のレビュー
review = "このアプリ、まあまあだけど使いにくい😄部分もあります。"

# レビューのクリーニング
cleaned_review = clean_review_with_gpt(review)

# クリーニング後のレビューを表示
print(cleaned_review)

In [ ]:
# dfのtextに対して，clean_review_with_gpt関数を適用．ポジティブな意見とネガティブな意見を分類しクリーニング，結果をポジティブな意見とネガティブな意見の二つの新たなカラムとして追加
def extract_reviews(cleaned_review):
    positive_review = re.search(r'ポジティブな意見:\s*(.*)', cleaned_review)
    negative_review = re.search(r'ネガティブな意見:\s*(.*)', cleaned_review)
    return positive_review.group(1) if positive_review else '', negative_review.group(1) if negative_review else ''

# まずは最初の10件に対してクリーニングを行う
#df[['positive_review', 'negative_review']] = df['text'].head(10).apply(lambda x: pd.Series(extract_reviews(clean_review_with_gpt(x))))

df[['positive_review', 'negative_review']] = df['text'].apply(lambda x: pd.Series(extract_reviews(clean_review_with_gpt(x))))

# データの確認
df

In [87]:
# クリーニング後のデータを保存
df.to_csv('data/processed/android_cleaned.csv', index=False)